In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
start_index = 19
end_index = 23
cam_name = 'cam2'

In [3]:
from services.label_service import get_map_not_zero_in_sphere

R = 10
map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, R, False)

map_size1=498154
map_size2=137163
map_size3=66483


In [4]:
import open3d as o3d

o3d.visualization.draw_geometries([map])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
from utils.pcd_utils import remove_stat_outlier

map, ind = remove_stat_outlier(map, nb_neighbors=25, std_ratio=5.0)
points2instances = points2instances[ind]

In [6]:
len(ind)

66333

In [7]:
o3d.visualization.draw_geometries([map])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [8]:
from services.label_service import build_o3d_voxel_pcd

voxel_size = 0.2
voxel_pcd, voxel_points2instances, voxel_trace = build_o3d_voxel_pcd(map, points2instances, start_index, end_index, voxel_size)

In [9]:
voxel_pcd

PointCloud with 2404 points.

In [10]:
voxel_points2instances.shape

(2404, 7)

In [12]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

voxel_points = np.asarray(voxel_pcd.points)
spatial_distance = cdist(voxel_points, voxel_points)

print("len(voxel_pcd.points) = {}".format(len(voxel_points)))
print("len(voxel_points2instances) = {}".format(len(voxel_points2instances)))

dist_voxel_points, masks = sam_label_distance(voxel_points2instances, spatial_distance, 2, 10)

len(voxel_pcd.points) = 2404
len(voxel_points2instances) = 2404


In [13]:
from utils.distances_utils import remove_isolated_points

dist, points = remove_isolated_points(dist_voxel_points, voxel_points)

In [14]:
dist.shape

(2404, 2404)

In [15]:
len(points)

2404

In [16]:
from services.normalized_cut_service import normalized_cut

T = 0.5
eigenval = 100
clusters = normalized_cut(dist, np.array([i for i in range(len(points))], dtype=int), T, eigenval)

ev = [-1.88592889e-04 -1.68810491e-04 -1.51310786e-04 ... -6.64647868e-04
 -2.62787429e-04 -5.81565037e-05]
mcut = 4.442811126956915e-05
ev = [-1.19058535e-16 -4.20021232e-15  7.85455350e-03  5.50383684e-02
 -1.69602732e-01  1.99159722e-16 -1.06377535e-01  2.31894832e-01
  1.50337794e-01 -4.53316647e-16  2.71361751e-16  3.43015162e-04
 -6.62897052e-01 -1.61310995e-01  3.63883577e-01 -1.32562537e-01
  4.99721562e-01  1.22611547e-16  2.71361751e-16 -2.73975260e-02
  1.54883272e-02  5.14242450e-17 -1.75872812e-02 -1.50658147e-02
  1.91466871e-02 -2.70212321e-15 -1.54919749e-02 -2.95440240e-17
 -4.25016821e-02 -2.45074895e-02  7.83412635e-03  4.61018350e-17
  1.88115266e-17  7.93592998e-17  4.00374479e-02  4.00374479e-02
  1.73955938e-02  3.28606615e-02  1.67977730e-02  5.46764187e-15
  2.72374301e-15 -3.71155098e-02 -1.06692590e-02 -1.06692590e-02
 -6.49151197e-02 -2.02389709e-17]
mcut = 0.9654812175025038


/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


ev = [0.01312467 0.01175116 0.00839368 ... 0.03708511 0.0183898  0.00420276]
mcut = 0.2629827288922841
ev = [6.83027803e-05 6.11547226e-05 4.37033163e-05 ... 5.08482254e-05
 1.93088443e-04 9.57017638e-05]
mcut = 1.575317071095271e-08
ev = [0.01314698 0.01177113 0.00840795 ... 0.00978386 0.03714787 0.01842106]
mcut = 0.19775618951878027
ev = [-0.01319682 -0.01181576 -0.00843983 ... -0.00982093 -0.03728871
 -0.01818399]
mcut = 0.37114627741632134
ev = [ 0.02816471  0.02533564 -0.0303464  ...  0.0362389  -0.00492733
  0.03882137]
mcut = 0.008537765800274925
ev = [ 0.06457388  0.06648499  0.03447599 -0.08027058 -0.05343755  0.0603749
 -0.0444072  -0.04593672  0.03345767  0.0494622   0.00475107 -0.00600274
 -0.03913494 -0.04288879 -0.03139332 -0.02847123 -0.02688154 -0.05031759
 -0.06168164 -0.06235623 -0.02818093  0.0661937   0.04350147 -0.01257803
  0.03914074  0.03499422  0.02168354 -0.04056362 -0.00642201 -0.023235
 -0.01845434 -0.06885353 -0.07630558 -0.07669717 -0.07262251 -0.05453645

/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 145 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/proj/lidar-labelling/services/normalized_cut_service.py:27: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)


ev = [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
mcut = inf
ev = [-0.03095447 -0.03967704 -0.04829323 -0.03765596 -0.02840153 -0.03595399
 -0.04233636 -0.04393195 -0.0

/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 40 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 81 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


In [17]:
len(clusters)

11

In [18]:
from utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_clustered = color_pcd_by_clusters_and_voxels(map, voxel_pcd, voxel_trace, clusters)

In [19]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [20]:
executed_id = 2
name = "labeled_clouds/clouds_{}_{}_R{}_voxel{}_T{}_eigenval{}_execute_id{}_outlier".format(start_index, end_index, R, voxel_size, T, eigenval, executed_id)
filename = name.replace(".", "l") + ".pcd"

o3d.io.write_point_cloud(filename, pcd_clustered)

True

In [22]:
import open3d as o3d

pcd_read = o3d.io.read_point_cloud("labeled_clouds/clouds_19_23_R10_voxel0l2_T0l5_eigenval100_execute_id2_outlier.pcd")
o3d.visualization.draw_geometries([pcd_read])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
